In [17]:
def match_cats(src, src_comp):

    from astropy.table import Table, vstack
    import numpy as np
    from copy import deepcopy

    t = Table()
    for col in src.getSchema().getNames():
        t[col] = deepcopy(src[col])

    t_comp = Table()
    for col in src_comp.getSchema().getNames():
        t_comp[col] = deepcopy(src_comp[col])
    
    import sys
    sys.path.insert(0,'/home/sr525/Python_Code')
    import match_lists

    ra = np.rad2deg(t["coord_ra"])
    dec = np.rad2deg(t["coord_dec"])

    ra_comp = np.rad2deg(t_comp["coord_ra"])
    dec_comp = np.rad2deg(t_comp["coord_dec"])
    print("Length of each catalogue:", len(t), len(t_comp))

    dist, inds = match_lists.match_lists(ra, dec, ra_comp, dec_comp, 1.0/3600.0)

    match = np.where( ( inds != len(ra_comp)) )[0]
    print("With match:", len(match))
    print("Difference in catalogue size:", len(ra)-len(ra_comp))

    t = t[match]
    t_comp = t_comp[inds[match]]
    
    return t, t_comp


In [2]:
def running_med(xs, ys, window, step_size=20):
    
    data = list(zip(xs, ys))
    data.sort(key = lambda t : t[0])
    [(xs), (ys)] = list(zip(*data))
    xs = np.array(xs)
    ys = np.array(ys)
    
    meds_out = []
    mads_out = []
    xs_out = []
    n = 0
    
    while n+window < len(ys):
        x = ys[n:n+window]
        xs_out.append(xs[n+window//2])
        med = np.median(x)
        meds_out.append(med)
        mads_out.append(mad(x, med))
        n += step_size
        
    return np.array(xs_out), np.array(meds_out), np.array(mads_out)

In [3]:
def mad(xs, med):
    
    return np.median(np.fabs(xs-med))

In [4]:
def flux_diff(t, t_comp, flux_col, g_type="div"):

    import matplotlib.pyplot as plt

    good = np.where((t[flux_col] == t[flux_col]) & (t_comp[flux_col]==t_comp[flux_col]))[0]

    xs = t[flux_col][good]
    if g_type == "div":
        ys = t[flux_col][good]/t_comp[flux_col][good]
    elif g_type == "sub":
        ys = t[flux_col][good] - t_comp[flux_col][good]

    med = np.median(ys)
    mad_ys = mad(ys, med)

    xs_out, meds_out, mads_out = running_med(xs, ys, 50)

    if g_type == "div":
        plt.axhline(1.0, ls = ":", color="k")
        plt.xlabel(flux_col + " compared, vanilla/4")
        
    elif g_type == "sub":
        plt.axhline(0.0, ls = ":", color="k")
        plt.xlabel(flux_col + " compared, vanilla - 4")
    plt.plot(xs, ys, "k.")
    plt.plot(xs_out, meds_out, "r.", label = "Running Median")
    plt.ylim(med-1.0*1.4826*mad_ys, med + 1.0*1.4826*mad_ys)
    plt.xscale("log")
    plt.axhline(med, label = "Median")
    plt.ylabel(flux_col)
    plt.legend()
    plt.show()

    print("Number of Sources:", len(t), "Median:", med, "Sigma MAD:", mad_ys*1.4826)
    
    return med, mad_ys*1.4826

In [5]:
def bright_flux_diff(t, t_comp, flux_col, g_type="div"):
    
    import numpy as np
    
    good = np.where((t[flux_col] == t[flux_col]))[0]
    thresh = np.percentile(t[flux_col][good], 70)
    ids = np.where((t[flux_col] > thresh))[0]
    
    print("All sources brighter than", thresh)
    med, sig_mad = flux_diff(t[ids], t_comp[ids], flux_col, g_type=g_type)
    
    return med, sig_mad

In [6]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import os

dir_comp = "/scratch/pprice/compression/ci_hsc/DATA/rerun/quantize4/"
butler_comp = dafPersist.Butler(inputs=dir_comp)

dir_van = "/scratch/pprice/compression/ci_hsc/DATA/rerun/vanilla/"
butler_van = dafPersist.Butler(inputs=dir_van)

dataDetails = {"filter": "HSC-R", "visit": 903334, "patch": "5,4", "tract": 0, "ccd": 100}

src_van = butler_van.get("forced_src", dataId=dataDetails)
src_comp = butler_comp.get("forced_src", dataId=dataDetails)

In [7]:
def visit_analysis(visit, filt, patch, tract, g_type="div"):
    
    ccds = []
    for filename in os.listdir(dir_comp + pointing + filt + "/tract" + str(tract)):
        if filename[11:17] == str(visit):
            ccds.append(int(filename[18:21]))
    i = 1
    for ccd in ccds:
        print("CCD:", ccd)
        dataDetails = {"filter": filt, "visit": visit, "patch": patch, "tract": tract, \
                       "ccd": ccd}
    
        src_van = butler_van.get("forced_src", dataId=dataDetails)
        src_comp = butler_comp.get("forced_src", dataId=dataDetails)

        t, t_comp = match_cats(src_van, src_comp)
        med, sig_mad = flux_diff(t, t_comp, flux_col, g_type=g_type)
    
        if i == 1:
            t_all = t
            t_all_comp = t_comp
        else:
            t_all = vstack([t_all, t])
            t_all_comp = vstack([t_all_comp, t_comp])
   
        i += 1

    print("All ccds in visit", visit)
    med_all, sig_mad_all = flux_diff(t_all, t_all_comp, flux_col, g_type=g_type)
    med_all_bright, sig_mad_all_bright = bright_flux_diff(t_all, t_all_comp, flux_col, \
                                                         g_type=g_type)
    
    return t_all, t_all_comp, med_all, sig_mad_all
  

In [19]:
from astropy.table import Table, vstack
import numpy as np

filt = "HSC-R"
visit = 903334
patch = "5,4"
tract = 0

if filt == "HSC-R":
    pointing = "00533/"
elif filt == "HSC-I":
    pointing = "00671/"

flux_col = "base_GaussianFlux_flux"
g_type = "sub"
visits = [903334, 903336, 903338, 903342, 903344, 903346]

i = 1
bad_cols = []
for visit in visits:
    print("Visit:", visit)
    ccds = []
    for filename in os.listdir(dir_comp + pointing + filt + "/tract" + str(tract)):
        if filename[11:17] == str(visit):
            ccds.append(int(filename[18:21])) 
    for ccd in ccds:
        src_van = butler_van.get("forced_src", dataId=dataDetails)
        src_comp = butler_comp.get("forced_src", dataId=dataDetails)

        t, t_comp = match_cats(src_van, src_comp)
 
        cols = t.columns
        cols = ["base_PsfFlux_flux"]
        print_out = True

        for col in cols:
            vals = t[col] - t_comp[col]
            good = np.where((vals == vals))[0]
            vals = vals[good]
            med_sub = np.median(vals)
            sig_mad_sub = 1.4826*mad(vals, med_sub)
    
            good = np.where((t[col] == t[col]))[0]
            med_in = np.median(t[col][good])
            sig_mad_in = 1.4826*mad(t[col][good], med_in)
    
            good = np.where((t_comp[col] == t_comp[col]))[0]
            med_comp = np.median(t_comp[col][good])
            sig_mad_comp = 1.4826*mad(t_comp[col][good], med_comp)
    
            vals_div = t[col]/t_comp[col]
            good_div = np.where((vals_div == vals_div))[0]
            vals_div = vals_div[good_div]
            med_div = np.median(vals_div)
            sig_mad_div = mad(vals_div, med_div)
    
            if np.fabs(1.0-med_in/med_comp) > 0.02 or \
               np.fabs(1.0-sig_mad_in/sig_mad_comp) > 0.02 or print_out:
    
                print(col)
                print("Subtracted:", med_sub, sig_mad_sub) 
                print("Divided:", med_div, sig_mad_div)
                print("Uncompressed:", med_in, sig_mad_in)
                print("Compressed:", med_comp, sig_mad_comp)
                print("Vanilla/Compressed:", med_in/med_comp, sig_mad_in/sig_mad_comp)
                print("|1-Van/comp|:", np.fabs(1.0-med_in/med_comp), np.fabs(1.0-sig_mad_in/sig_mad_comp))
                print("\n")
            
                bad_cols.append(col)
    
    

Visit: 903334
Length of each catalogue: 1105 1161
With match: 1069
Difference in catalogue size: -56
base_PsfFlux_flux
Subtracted: 0.0646156271359 10.2664823902
Divided: 1.0000140505 0.0110141265402
Uncompressed: 435.313542351 426.27057262
Compressed: 431.735917621 420.54953699
Vanilla/Compressed: 1.00828660434 1.01360371401
|1-Van/comp|: 0.00828660434412 0.0136037140135


Length of each catalogue: 1105 1161
With match: 1069
Difference in catalogue size: -56
base_PsfFlux_flux
Subtracted: 0.0646156271359 10.2664823902
Divided: 1.0000140505 0.0110141265402
Uncompressed: 435.313542351 426.27057262
Compressed: 431.735917621 420.54953699
Vanilla/Compressed: 1.00828660434 1.01360371401
|1-Van/comp|: 0.00828660434412 0.0136037140135


Length of each catalogue: 1105 1161
With match: 1069
Difference in catalogue size: -56
base_PsfFlux_flux
Subtracted: 0.0646156271359 10.2664823902
Divided: 1.0000140505 0.0110141265402
Uncompressed: 435.313542351 426.27057262
Compressed: 431.735917621 420.549536

In [12]:
print(len(bad_cols))
print(set(bad_cols))
print(len(set(bad_cols)))

221
{'base_CircularApertureFlux_35_0_flux', 'ext_photometryKron_KronFlux_flux', 'base_CircularApertureFlux_17_0_flux', 'base_GaussianFlux_flux', 'base_CircularApertureFlux_9_0_flux', 'ext_photometryKron_KronFlux_fluxSigma', 'base_TransformedShape_yy', 'objectId', 'base_CircularApertureFlux_17_0_fluxSigma', 'base_CircularApertureFlux_70_0_flux', 'ext_photometryKron_KronFlux_radius', 'base_TransformedShape_xx', 'base_CircularApertureFlux_50_0_flux'}
13


In [ ]:
from astropy.table import Table, vstack

filt = "HSC-R"
visit = 903334
patch = "5,4"
tract = 0

if filt == "HSC-R":
    pointing = "00533/"
elif filt == "HSC-I":
    pointing = "00671/"


flux_col = "base_GaussianFlux_flux"
g_type = "sub"
visits = [903334, 903336, 903338, 903342, 903344, 903346]

for flux_col in set(bad_cols):
    print(flux_col)
    i = 1
    for visit in visits:
        print(visit)
        t, t_comp, med, sig_mad = visit_analysis(visit, filt, patch, tract, g_type=g_type)
    
        if i == 1:
            t_all = t
            t_all_comp = t_comp
        else:
            t_all = vstack([t_all, t])
            t_all_comp = vstack([t_all_comp, t_comp])
        i += 1
        
    print("All sources in all visits")        
    med_all, sig_mad_all = flux_diff(t_all, t_all_comp, flux_col, g_type=g_type)
    med_all_bright, sig_mad_all_bright = bright_flux_diff(t_all, t_all_comp, flux_col, \
                                                      g_type=g_type)
